# Aquisição e Análise de Medições

A classe `MDMS` utiliza as configurações definidas no arquivo `config.toml` para inicializar o motor de aquisição para preparar o ambiente para geração das medições sintéticas via OpenDSS.

O sistema realiza a geração de dados sintéticos para:
- Potência ativa (P)  
- Potência reativa (Q)  
- Magnitudes de tensão (|V|)





In [0]:
# Widgets
dbutils.widgets.text("sub", "TAQ", "Subestação")
dbutils.widgets.text("feeder", "TAQ03", "Alimentador")
dbutils.widgets.text("uni_tr_mt", "34102241", "Rede BT")
dbutils.widgets.text("random_state", "42", "Semente Aleatória")
sub = dbutils.widgets.get("sub")
feeder = dbutils.widgets.get("feeder")
uni_tr_mt = dbutils.widgets.get("uni_tr_mt")
random_state = dbutils.widgets.get("random_state")
if random_state == "":
    random_state = None
else:
    random_state = int(random_state)

In [0]:
import os
import sys
current_path = os.getcwd()
parent_path = os.path.dirname(current_path)
sys.path.append(parent_path)

In [0]:
# MDMS engine
from Utils.MDMS import MDMS, config
mdms = MDMS(sub=sub, feeder=feeder, uni_tr_mt=uni_tr_mt, random_state=random_state)
meas = mdms.acquire()

In [0]:
display(mdms.irregs.drop(columns=["theft_kw", "measured_kw"]))

In [0]:
fig = mdms.plot_ntl()

In [0]:
fig = mdms.plot_topology(
    irreg_buses = mdms.irregs["bus_id"].tolist()
)

# Treinamento do LR (LR)

Treinamento de um modelo MLP voltado à previsão das magnitudes de tensão em um sistema elétrico de baixa tensão. 



In [0]:
from sklearn.linear_model import LinearRegression
from Models.LinearRegression import Training
model = LinearRegression()
inputs, outputs = mdms.training.get("inputs"), mdms.training.get("outputs") 
engine = Training(
    model=model, 
    inputs=inputs, 
    outputs=outputs,
    test_size=0.3,
    random_state=random_state
)
model = engine.train()
mse, r2 = engine.evaluate_model()
print(f'MSE: {mse:.6f}, R²: {r2:.6f}')

In [0]:
engine.fig_histogram

In [0]:
engine.fig_boxplot

In [0]:
from Models.LinearRegression import Detector
inputs, outputs = mdms.inference.get("inputs"), mdms.inference.get("outputs") 
detector = Detector(model=model, inputs=inputs, outputs=outputs)
deltaW = detector.execute()

In [0]:
display(deltaW)
    
    

In [0]:
display(mdms.irregs.drop(columns=["measured_kw", "theft_kw"]))

In [0]:
import numpy as np
import pandas as pd
df = pd.merge(left=mdms.irregs[["bus_id", "num_phases", "kw_by_phase", "pf", "kwh_theft"]], right=deltaW, on="bus_id", how="right").dropna()
df["error"] = np.round(abs((df["kwh_theft"] - df["deltaW"])/df["kwh_theft"]),2)
display(df)

In [0]:
from Utils.PlotFunctions import boxplot
fig = boxplot(np.asarray(detector.residues_list)/2.50)

In [0]:
from Utils.PlotFunctions import plot_inputs, plot_outputs
i = -1
outputs_ = detector.outputs_.query("updt==1 and r2<0.5")
inputs_ = detector.inputs_.iloc[outputs_.index]
y_pred = outputs_.iloc[i].outputs
y_true = outputs_.iloc[i].targets
y_optim = outputs_.iloc[i].optimized
x_input = inputs_.iloc[i].inputs
x_optim = inputs_.iloc[i].optimized
columns = inputs.columns
fig1 = plot_inputs(
    columns = columns, 
    x_input = x_input, 
    x_optim = x_optim
)

In [0]:
fig2 = plot_outputs(
    columns = outputs.columns, 
    y_pred = y_pred, 
    y_true = y_true, 
    y_optim = y_optim
)